In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize

df = pd.read_csv("/content/FantasyPros_2023_Draft_ALL_Rankings - FantasyPros_2023_Draft_ALL_Rankings (1).csv")
df = df.drop(columns=['TIERS','TEAM', 'PLAYER NAME'])
df = df.rename(columns={"Value": "True Value", "STD.DEV": "STD","RK":"Player ID"})
df["Player ID"] -= 1

FileNotFoundError: ignored

In [ ]:
n=5
m=len(df.index)
print(m)
sigma = list(range(n))
constraints = {"QB":1, "RB":2, "WR": 2, "TE": 1, "K":1, "DST":1}
category_items = {"QB":list(df[df['Position'].isin(["QB"])]["Player ID"]), "RB":list(df[df['Position'].isin(["RB"])]["Player ID"]), "WR": list(df[df['Position'].isin(["WR"])]["Player ID"]), "TE": list(df[df['Position'].isin(["TE"])]["Player ID"]), "K":list(df[df['Position'].isin(["K"])]["Player ID"]), "DST":list(df[df['Position'].isin(["DST"])]["Player ID"])}


461


{'QB': [346,
  347,
  348,
  349,
  350,
  351,
  352,
  353,
  354,
  355,
  356,
  357,
  358,
  359,
  360,
  361,
  362,
  363,
  364,
  365,
  366,
  367,
  368,
  369,
  370,
  371,
  372,
  373,
  374,
  375,
  376,
  377,
  378,
  379,
  380,
  381,
  382,
  383,
  384,
  385,
  386,
  387,
  388,
  389,
  390,
  391,
  392,
  393,
  394,
  395],
 'RB': [218,
  219,
  220,
  221,
  222,
  223,
  224,
  225,
  226,
  227,
  228,
  229,
  230,
  231,
  232,
  233,
  234,
  235,
  236,
  237,
  238,
  239,
  240,
  241,
  242,
  243,
  244,
  245,
  246,
  247,
  248,
  249,
  250,
  251,
  252,
  253,
  254,
  255,
  256,
  257,
  258,
  259,
  260,
  261,
  262,
  263,
  264,
  265,
  266,
  267,
  268,
  269,
  270,
  271,
  272,
  273,
  274,
  275,
  276,
  277,
  278,
  279,
  280,
  281,
  282,
  283,
  284,
  285,
  286,
  287,
  288,
  289,
  290,
  291,
  292,
  293,
  294,
  295,
  296,
  297,
  298,
  299,
  300,
  301,
  302,
  303,
  304,
  305,
  306,
  307,
  308,


In [ ]:
def generate_noisy_values(N,truth_data):
  values = np.ones((N,len(truth_data)))
  for n in range(N):
    for player_id in range(len(truth_data)):
      value = np.random.normal(pd.to_numeric(truth_data["True Value"][player_id]), pd.to_numeric(truth_data["STD"][player_id]), 1)[0]
      if(value<0):
        value = 0
      values[n,player_id] = value
  values = normalize(values, axis=1, norm='l1')
  return values

In [ ]:
V_i = generate_noisy_values(n,df)

NameError: ignored

In [ ]:
def envy_graph(A, v_agents):
  n = len(A)
  G = {i: [] for i in range(n)}

  for i in range(n):
      for j in range(n):
          if i != j and sum(v_agents[i][list(A[i])]) < sum(v_agents[j][list(A[i])]):
              G[i].append(j)

  return G

def topological_sort(graph):
  # Compute in-degree for each node in the graph
  in_degree = {node: 0 for node in graph}
  for node in graph:
      for neighbor in graph[node]:
          in_degree[neighbor] += 1
  
  # Find nodes with in-degree 0
  zero_in_degree = [node for node in graph if in_degree[node] == 0]
  
  # Perform topological sort
  sorted_nodes = []
  while zero_in_degree:
      node = zero_in_degree.pop()
      sorted_nodes.append(node)
      for neighbor in graph[node]:
          in_degree[neighbor] -= 1
          if in_degree[neighbor] == 0:
              zero_in_degree.append(neighbor)
  
  if len(sorted_nodes) != len(graph):
      # Graph has a cycle
      raise ValueError('Graph is not acyclic')
  
  return sorted_nodes

In [ ]:
import random

def Greedy_Round_Robin_Different_Valuations(m, n, V_i, sigma, numallocsperhead):

  bundles = [set() for i in range(n)]
  goods = set(m)
  # Initialize the current agent index and round number
  i = 0
  t = 0
  # Iterate until all goods are allocated
  while t<(numallocsperhead*n):
    # Select the next agent
    i = sigma[t % n]
    # Increment the round number
    t += 1         

    # Calculate the value of each unallocated good for this agent
    values = []
    for j in goods:
        values.append(V_i[i][j])
        
    # Find the maximum value for this agent
    max_value = max(values)

    # Find the indices of the goods that have the maximum value for this agent
    max_indices = [j for j in goods if V_i[i][j] == max_value]

    # Select a random index from the max indices
    index = random.choice(max_indices)

          
    # Add the selected good to the current agent's bundle
    bundles[i].add(index)
        
    # Remove the selected good from the set of unallocated goods
    goods.remove(index)        
    
  return(bundles)

In [ ]:
def EF1(n, V_i, categories, category_items):
  # Initialize allocation A0 = (A0_1, . . . , A0_n) with A0_i ← ∅ for each agent i ∈ [n].
  A = [set() for i in range(n)]

  sigma = list(range(n))
  # Iterate until convergence
  for h in categories:
      # Call Greedy-Round-Robin to get the allocation for this category
    players = category_items[h]
    B = Greedy_Round_Robin_Different_Valuations(players, n, V_i, sigma, constraints[h])

    # Update the allocation
    for i in range(n):
        A[i].update(B[i])
        
    G = envy_graph(A, V_i)
    sigma = topological_sort(G)

  return(A)

In [ ]:
allocations = EF1(5, V_i, constraints, category_items)

In [ ]:
def evaluate_allocation(values, allocation,N,truth_data):
  sw = 0
  norm_alloc = 0
  for n in range(N):
    alloc_value = 0
    for i in allocation[n]:
      alloc_value = alloc_value + values[n,i]
    print(n, ": ",  alloc_value)
    print("Normalized: ",  alloc_value*len(truth_data)/len(allocation[n]))
    norm_alloc = norm_alloc + alloc_value*len(truth_data)/len(allocation[n])
    sw = sw + alloc_value
  return sw, norm_alloc

In [ ]:
sw = evaluate_allocation(V_i, allocations,n, df)
print("Social Welfare: ", sw)

0 :  0.023625141038909555
Normalized:  1.3613987523671631
1 :  0.024254815778378364
Normalized:  1.3976837592290532
2 :  0.023261071845842037
Normalized:  1.3404192651166473
3 :  0.024239454571522417
Normalized:  1.3967985696839793
4 :  0.023286662102837776
Normalized:  1.341893903676027
Social Welfare:  (0.11866714533749016, 6.8381942500728705)
